In [2]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

In [3]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import folium
from package import storage
from typing import Optional

In [4]:
stops_df = storage.read_df('../data/cleaned/stops.csv')
stop_times_df = storage.read_df('../data/cleaned/stop_times.csv')
trips_df = storage.read_df('../data/cleaned/trips.csv')
# routes_df = storage.read_df('../data/cleaned/routes.csv')

In [5]:
def plot_route(route_id: str, m: Optional[folium.Map] = None) -> folium.Map:
    associated_trips = trips_df[trips_df["route_id"] == route_id]
    trip = associated_trips.iloc[0]
    associated_stops = stop_times_df[stop_times_df["trip_id"] == trip["trip_id"]]
    associated_stops = associated_stops.merge(stops_df, on="stop_id")

    m = (
        folium.Map(
            location=[
                associated_stops["stop_lat"].mean(),
                associated_stops["stop_lon"].mean(),
            ],
            zoom_start=12,
        )
        if m is None
        else m
    )

    folium.PolyLine(
        associated_stops[["stop_lat", "stop_lon"]].values.tolist(),
        color="red",
        weight=2.5,
        opacity=1,
    ).add_to(m)
    return m


def plot_stop(stop_id: str, m: Optional[folium.Map] = None) -> folium.Map:
    stop = stops_df[stops_df["stop_id"] == stop_id].iloc[0]
    m = (
        folium.Map(location=[stop["stop_lat"], stop["stop_lon"]], zoom_start=12)
        if m is None
        else m
    )
    folium.Marker([stop["stop_lat"], stop["stop_lon"]], popup=stop["stop_name"]).add_to(
        m
    )
    return m


In [7]:
m = plot_route("100016_1_C")
plot_stop("880", m)